In [1]:
### By Olek Yardas [oyardas2]
### and Dreycen Foiles [dfoiles2]

### Work breakdown
### Olek: part 1 algorithm, derivations, and results
### Dreycen : part 2 algorithm, derivations, and results

In [2]:
library(HMM)
library(mclust)

Package 'mclust' version 5.4.10
Type 'citation("mclust")' for citing this R package in publications.



In [3]:
dim(faithful)
head(faithful)

[1] 272   2

,eruptions,waiting
,<dbl>,<dbl>
1,3.600,79
2,1.800,54
3,3.333,74
4,2.283,62
5,4.533,85
6,2.883,55


# Part 1

### Derivations

Marginal likelihood function

$$\begin{aligned}
& \prod_{i=1}^n  p(x_i \mid p_{1:G}, \mu_{1:G}, \Sigma) \\
= & \prod_{i=1}^n  \big[   p_1 N(x_i; \mu_1, \Sigma) + \cdots + p_G
N(x_i; \mu_G, \Sigma) \big ]\\
= & \prod_{i=1}^n  \Big [ p_1  \frac{\exp  ( - \frac{1}{2} (x-
\mu_1)^t \Sigma^{-1} (x - \mu_1)  )}{\sqrt{(2 \pi)^p | \Sigma| }}
+ \cdots + p_G \frac{\exp  ( - \frac{1}{2} (x- \mu_G)^t \Sigma^{-1} (x -
\mu_G)  )}{\sqrt{(2 \pi)^p | \Sigma| }} \Big ]
\end{aligned}$$

Complete likelihood function

$$\begin{aligned}
& \prod_{i=1}^n  p(x_i, Z_i \mid p_{1:G}, \mu_{1:G}, \Sigma) \\
= & \prod_{i=1}^n \prod_{k=1}^G   \Big [ p_k  \frac{\exp  ( -
\frac{1}{2} (x- \mu_k)^t \Sigma^{-1} (x - \mu_k) )}{\sqrt{(2 \pi)^p |
\Sigma| }} \Big ]^{\mathbb{1}_{\{Z_i = k \}}}
\end{aligned}$$

Distribution of $\mathbb{Z}_i$ at the E-step

$$p(x_n, \mathbb{Z}_i = k | p_{1:G}, \mu_{1:G}, \Sigma) = \frac{p_k \mathcal{N}(x_n|\mu_k, \Sigma)}{\sum_j p_j \mathcal{N}(x_n | \mu_j, \Sigma)}$$

where $\mathcal{N}(x_n|\mu_k, \Sigma) = p_k  \frac{\exp  ( -
\frac{1}{2} (x- \mu_k)^t \Sigma^{-1} (x - \mu_k) )}{\sqrt{(2 \pi)^p |
\Sigma| }}$

M-step objective function

$$\begin{aligned}
g(p_{1:G}, \mu_{1:G}, \Sigma) =&  \mathbb{E} \log
\prod_{i=1}^n  p(x_i, Z_i \mid p_{1:G}, \mu_{1:G}, \Sigma)\\
=& \mathbb{E} \sum_{i=1}^n \sum_{k=1}^G  \mathbb{1}_{\{Z_i = k \}}
\log  \Big [ p_k  \frac{\exp  ( - \frac{1}{2} (x- \mu_k)^t \Sigma^{-1}
(x - \mu_k) )}{\sqrt{(2 \pi)^p | \Sigma| }} \Big ] \\
=& \sum_{i=1}^n \sum_{k=1}^G  p_{ik}  \log  \Big [
p_k  \frac{\exp  ( - \frac{1}{2} (x- \mu_k)^t \Sigma^{-1} (x - \mu_k)
)}{\sqrt{(2 \pi)^p | \Sigma| }} \Big ]
\end{aligned}$$

where the last step is due to the fact that $\mathbb{E} [ \mathbb{1}_{\{Z_i = k \}}] =
\mathbb{P}(Z_i = k) = p_{ik}.$


Updating formula for $p_{1:G}, \mu_{1:G}, \Sigma$

Taking the derivative of the previous function w.r.t $p_j$, and adding a Lagrange multiplier and setting to zero, we get
$$
\begin{align}
\frac{\partial}{\partial p_j} \sum_{i=1}^n \sum_{k=1}^G  p_{ik}  \log p_k + \lambda(1 - \sum_k p_k) = & \sum_{i=1}^np_{ij} \frac{1}{p_{j}} - \lambda \\
=& 0
\end{align}
$$
Rearranging, we get $p_j = \frac{\sum_{i=1}^n p_{ij}}{\lambda}$
Taking the derivative of our original expression w.r.t. $\lambda$, we get $\sum_k p_k = 1$
Substituting the previous result into this equation and rearranging, we get
$$\sum_{i=1}^n \sum_k p_{ik} = \lambda$$. Since $\sum_k p_{ik} = 1$, this becomes $\lambda = n$. So our final expression is $$\boxed{p_k = \frac{1}{n} \sum_{i=1}^n p_{ik}}$$

Following a similar procedure for $\mu_k$ and $\Sigma_k$, we have

$$\boxed{\mu_k^\text{new} = \frac{\sum_i p_{ik} x_i}{\sum_i p_{ik}}}$$
and 
$$\boxed{\Sigma_k = \frac{\sum_i p_{ik} (x_i - \mu_k^\text{new})(x_n - \mu_k^\text{new})^T}{\sum_i p_{ik}}}$$

In [4]:
mygauss <- function(data, mean, Sigma){
    p = nrow(Sigma)
    svd_obj = svd(Sigma)
    D = svd_obj$d
    U = svd_obj$u
    V = svd_obj$v
    Dt = diag(1/sqrt(D))
    A = t(data)
    Sinv = U %*% Dt^2 %*% t(U) 

    #xt = Dt %*% t(U) %*% data
    #mt = Dt %*% t(U) %*% mean
    #a = t(xt-mt) %*% Sinv %*% (xt-mt)
    a = (A - mean) * (Sinv %*% (A - mean))
    a = colSums(a)
    return (exp(-0.5 * a)/((2*pi)^(p/2) * (det(Sigma))^0.5))
}

Estep <- function(data, G, para){
    # Your Code
    prob <- para$prob
    mu <- para$mean
    Sigma <- para$Sigma
    loglik <- para$loglik
    n <- nrow(data)
    # need to come up with a way to evalue multivariate gaussian distrbutions,
    # but let's focus on getting the algorithm correct right now.
    # assume we have G means, G Sigmas, G pis
    p = matrix(0, n, G)
    for (k in 1:G){
        p[,k] = prob[k] * mygauss(data, mu[,k], Sigma)
    }
    #p = prob * mygauss(data, mu, Sigma)
    s = rowSums(p)
    p = p / s
  
    # Return the n-by-G probability matrix
    return(p)
  }

Mstep <- function(data, G, para, post.prob){ 
    # Your Code
    # Return the updated parameters
    para$prob = colSums(post.prob) / nrow(post.prob)
    mu = t(data) %*% post.prob
    #mu_sum = colSums(mu)
    mu_sum = colSums(post.prob)
    n = nrow(mu)
    for (k in 1:n){
        mu[k,] = mu[k,] / mu_sum
    }
    para$mean = mu
    Sigma = matrix(0, n, n)
    # This loop is giving the first step Sigma very close agreement with the 
    # R method, but not exact agreement. 
    for (k in 1:G) {
        x = t(data) - mu[,k]
        P = diag(post.prob[,k])
        Sigma = Sigma + (x %*% P %*% t(x))
    }
    
    para$Sigma = Sigma / nrow(post.prob)
    
    return (para)
  }

loglik <- function(data, G, para){
    # compute loglikelihood
    prob <- para$prob
    mu <- para$mean
    Sigma <- para$Sigma

    n = nrow(data)
    ll = matrix(0, n, G)
    for (k in 1:G) {
        ll[,k] = prob[k] * mygauss(data, mu[,k], Sigma)
    }
    #for (k in 1:G) {
    #    ll[,k] = prob[k] * log(ll[,k])
    #}
    ll = log(rowSums(ll))
    ll = sum(ll)
    
    return (ll)
}

myEM <- function(data, itmax, G, para){
  # itmax: number of of iterations
  # G:     number of components
  # para:  list of (prob, mean, Sigma, loglik)
  d = as.matrix(data)
  for(t in 1:itmax){
    post.prob <- Estep(d, G, para)
    para <- Mstep(d, G, para, post.prob)
  }
  
  # update para$loglik   
  para$loglik = loglik(d, G, para)
  
  return(para)
}

options(digits=8)
options()$digits

[1] 8

In [5]:
n <- nrow(faithful)
G <- 2
set.seed(7568)  # replace 234 by the last 4-dig of your University ID
gID <- sample(1:G, n, replace = TRUE)
Z <- matrix(0, n, G)
for(k in 1:G)
  Z[gID == k, k] <- 1 
ini0 <- mstep(modelName="EEE", faithful , Z)$parameters

para0 <- list(prob = ini0$pro, 
              mean = ini0$mean,
              Sigma = ini0$variance$Sigma, 
              loglik = NULL)

### G = 2

In [6]:
my_res <- myEM(d=faithful, itmax=20, G=G, para=para0)

Rout <- em(modelName = "EEE", data = faithful,
           control = emControl(eps=0, tol=0, itmax = 20), 
           parameters = ini0)

prob = Rout$para$pro
mean = Rout$para$mean
Sigma = Rout$para$variance$Sigma
ll = Rout$loglik

print('Error between myEM and mclust em')
print('Weights')
(prob - my_res$prob) / prob
print('Means')
(mean - my_res$mean) / mean
print('Covariance Matrix')
(Sigma - my_res$Sigma) / Sigma
print('Log likelihood')
(ll - my_res$loglik) / ll

[1] "Error between myEM and mclust em"
[1] "Weights"


[1] -8.5582389e-16  1.0384598e-15

[1] "Means"


eruptions,3.2148647e-15,-5.2619209e-15
waiting,-3.9712110e-15,3.0377230e-15


[1] "Covariance Matrix"


,eruptions,waiting
eruptions,-1.8988099e-15,1.2824590e-15
waiting,1.2824590e-15,4.6426469e-16


[1] "Log likelihood"


[1] 5.2886106e-16

### G = 3

In [7]:
n <- nrow(faithful)
G <- 3
set.seed(7568)  # replace 234 by the last 4-dig of your University ID
gID <- sample(1:G, n, replace = TRUE)
Z <- matrix(0, n, G)
for(k in 1:G)
  Z[gID == k, k] <- 1 
ini0 <- mstep(modelName="EEE", faithful , Z)$parameters

para0 <- list(prob = ini0$pro, 
              mean = ini0$mean,
              Sigma = ini0$variance$Sigma, 
              loglik = NULL)

In [8]:
my_res <- myEM(d=faithful, itmax=20, G=G, para=para0)

Rout <- em(modelName = "EEE", data = faithful,
           control = emControl(eps=0, tol=0, itmax = 20), 
           parameters = ini0)

prob = Rout$para$pro
mean = Rout$para$mean
Sigma = Rout$para$variance$Sigma
ll = Rout$loglik

print('Error between myEM and mclust em')
print('Weights')
(prob - my_res$prob) / prob
print('Means')
(mean - my_res$mean) / mean
print('Covariance Matrix')
(Sigma - my_res$Sigma) / Sigma
print('Log likelihood')
(ll - my_res$loglik) / ll

[1] "Error between myEM and mclust em"
[1] "Weights"


[1] -2.8827498e-15 -3.4321796e-16  3.9311910e-15

[1] "Means"


eruptions,2.7246403e-15,-3.0971074e-15,1.1939944e-16
waiting,1.9787096e-16,4.2302086e-16,-1.9288796e-15


[1] "Covariance Matrix"


,eruptions,waiting
eruptions,-6.4368810e-15,-3.6078423e-15
waiting,-3.6078423e-15,-3.2117496e-16


[1] "Log likelihood"


[1] -1.7630869e-16

# Problem 2

In [9]:
myBW = function(x, para, n.iter = 100){
  # Input:
  # x: T-by-1 observation sequence
  # para: initial parameter value
  # Output updated para value (A and B; we do not update w)
  
  for(i in 1:n.iter){
    para = BW.onestep(x, para)
  }
  return(para)
}

In [10]:
BW.onestep = function(x, para){
    # Input: 
    # x: T-by-1 observation sequence
    # para: mx, mz, and current para values for
    #    A: initial estimate for mz-by-mz transition matrix
    #    B: initial estimate for mz-by-mx emission matrix
    #    w: initial estimate for mz-by-1 initial distribution over Z_1
    # Output the updated parameters after one iteration
    # We DO NOT update the initial distribution w

    T = length(x)
    mz = para$mz
    mx = para$mx
    A = para$A
    B = para$B
    w = para$w
    alp = forward.prob(x, para)
    beta = backward.prob(x, para)
    

    myGamma = array(0, dim=c(mz, mz, T-1))
    myGamma_i = matrix(0, T, mz)
    #######################################
    ## YOUR CODE: 
    ## Compute gamma_t(i,j) P(Z[t] = i, Z[t+1]=j), 
    for (t in 1:(T-1)) {
        for (i in 1:mz) {
          for (j in 1:mz) {
            myGamma[i, j, t] = alp[t,i] * A[i,j] * B[j, x[t+1]] * beta[(t+1),j]
          }
        }
      }
    
    for (t in 1:(T-1)){
        for (i in 1:mz){
            myGamma_i[t,i] = sum(myGamma[i,,t] )
        }
    }

    for (i in 1:mz){
            myGamma_i[T,i] = sum(myGamma[,i,T-1])
        }
    

    ## which are stored in an array, myGamma
    #######################################

    # M-step for parameter A
    #######################################
    ## YOUR CODE: 
    
    A = apply(myGamma, MARGIN=c(1,2), sum)
    A = A/rowSums(A)
    
    #######################################

    # M-step for parameter B
    #######################################
    ## YOUR CODE: 

    B[] = 0
    for (i in 1:mz) {
        for (l in 1:mx) {
          idx = which(x == l)
          B[i,l] = sum(myGamma_i[idx,i]) / sum(myGamma_i[,i])
        }
      }

    #######################################

    para$A = A
    para$B = B
    return(para)
}

In [11]:
forward.prob = function(x, para){
  # Output the forward probability matrix alp 
  # alp: T by mz, (t, i) entry = P(x_{1:t}, Z_t = i)
  T = length(x)
  mz = para$mz
  A = para$A
  B = para$B
  w = para$w
  alp = matrix(0, T, mz)
  
  # fill in the first row of alp
  alp[1, ] = w * B[, x[1]]
  # Recursively compute the remaining rows of alp
  for(t in 2:T){
    tmp = alp[t-1, ] %*% A
    alp[t, ] = tmp * B[, x[t]]
    }
  return(alp)
}

backward.prob = function(x, para){
  # Output the backward probability matrix beta
  # beta: T by mz, (t, i) entry = P(x_{1:t}, Z_t = i)
  T = length(x)
  mz = para$mz
  A = para$A
  B = para$B
  w = para$w
  beta = matrix(1, T, mz)

  # The last row of beta is all 1.
  # Recursively compute the previous rows of beta
  for(t in (T-1):1){
    tmp = as.matrix(beta[t+1, ] * B[, x[t+1]])  # make tmp a column vector
    beta[t, ] = t(A %*% tmp)
    }
  return(beta)
}

In [12]:
myViterbi = function(x, para){
    # Output: most likely sequence of Z (T-by-1)
    T = length(x)
    mz = para$mz
    A = para$A
    B = para$B
    w = para$w
    log.A = log(A)
    log.w = log(w)
    log.B = log(B)

    # Compute delta (in log-scale)
    delta = matrix(0, T, mz) 
    # fill in the first row of delta
    delta[1, ] = log.w + log.B[, x[1]]

    #######################################
    ## YOUR CODE: 
    ## Recursively compute the remaining rows of delta
    for (t in 2:T){
        for (i in 1:mz){
            jVec = rep(0, mz)
            for (j in 1:mz){
                jVec[j] = delta[t-1,j] + log.A[j,i]
            }
            delta[t,i] = max(jVec) + log.B[i, x[t]]
        }
            
    }

    #######################################

    # Compute the most prob sequence Z
    Z = rep(0, T)
    # start with the last entry of Z
    Z[T] = which.max(delta[T, ])
    
    for (t in (T-1):1){
        Z[t] = which.max(delta[t,] + log.A[,Z[t+1]])
    }

    #######################################
    ## YOUR CODE: 
    ## Recursively compute the remaining entries of Z
    #######################################

    return(Z)
}

In [13]:
data = scan("coding4_part2_data.txt")

mz = 2
mx = 3
ini.w = rep(1, mz); ini.w = ini.w / sum(ini.w)
ini.A = matrix(1, 2, 2); ini.A = ini.A / rowSums(ini.A)
ini.B = matrix(1:6, 2, 3); ini.B = ini.B / rowSums(ini.B)
ini.para = list(mz = 2, mx = 3, w = ini.w,
                A = ini.A, B = ini.B)

myout = myBW(data, ini.para, n.iter = 100)

myout.Z = myViterbi(data, myout)
myout.Z[myout.Z==1] = 'A'
myout.Z[myout.Z==2] = 'B'

In [14]:
library(HMM)
hmm0 =initHMM(c("A", "B"), c(1, 2, 3),
              startProbs = ini.w,
              transProbs = ini.A, 
              emissionProbs = ini.B)
Rout = baumWelch(hmm0, data, maxIterations=100, delta=1E-9, pseudoCount=0)
Rout.Z = viterbi(Rout$hmm, data)

In [15]:
options(digits=8)
options()$digits

[1] 8

In [16]:
myout$A
Rout$hmm$transProbs

0.49793938,0.50206062
0.44883431,0.55116569


,A,B
A,0.49793938,0.50206062
B,0.44883431,0.55116569


In [17]:
myout$B
Rout$hmm$emissionProbs

0.22159897,0.20266127,0.57573976
0.34175148,0.17866665,0.47958186


,1,2,3
A,0.22159897,0.20266127,0.57573976
B,0.34175148,0.17866665,0.47958186


In [18]:
cbind(Rout.Z, myout.Z)[c(1:10, 180:200), ]
sum(Rout.Z != myout.Z)

Rout.Z,myout.Z
A,A
A,A
A,A
A,A
A,A
A,A
A,A
B,B
A,A
A,A


[1] 0